In [ ]:
#!pip install webdriver-manager

# 크롤링

## 패키지 설치

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from time import sleep
import random
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

## 로그인

In [ ]:
username= 'username'
password='password'

login_url='https://ewportal.ewha.ac.kr/login.do'
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.implicitly_wait(3)
driver.get(login_url)
driver.maximize_window()

#로그인
a=driver.find_element(By.CLASS_NAME,'box-con')
username_input=a.find_element(By.CLASS_NAME,'login-input').find_element(By.NAME, "loginId")
username_input.send_keys(username)
password_input=a.find_element(By.CLASS_NAME,'login-input').find_element(By.NAME, "userPass")
password_input.send_keys(password)
submit_button = a.find_element(By.CLASS_NAME,'login-btn')
submit_button.click()

In [ ]:
#분실물 게시판으로 이동
lost_url='https://ewportal.ewha.ac.kr/portal/menu/goPage.do?anyoneId=MU7d6ab73e12fd4287bdcd127e26c3b214&nodeId=MU7d6ab73e12fd4287bdcd127e26c3b214&nodeType=SIDE_MENU'
driver.get(lost_url)

In [ ]:
title=[];period=[];postdate=[];modifydate=[]
person=[];view=[];category=[];details=[]
section=[];process=[];phonenum_yn=[];phonenum=[]
place=[];place_details=[];state=[]
image=[];text=[]

## 창 하나씩 들어가기

In [ ]:
def page_crawling():
    a=driver.find_elements(By.CLASS_NAME,'text-left')
    #print(a)
    time.sleep(1)
    for i in range(1,len(a)):
        p='//*[@id="listForm"]/table/tbody/tr['+str(i)+']/td[3]/div/span[1]/a'
        aa=driver.find_element(By.XPATH,p)
        driver.execute_script('arguments[0].click();',aa)


        #btn=a[i].find_elements(By.TAG_NAME,'a')
        #driver.execute_script('arguments[0].click();',btn[0])

        pyo=driver.find_elements(By.CLASS_NAME, 'sw-board-td.sw-board-ft')

        title.append(pyo[0].text)
        period.append(pyo[1].text)
        postdate.append(pyo[2].text)
        modifydate.append(pyo[3].text)
        person.append(pyo[4].text)
        view.append(pyo[5].text)
        category.append(pyo[6].text)
        details.append(pyo[7].text)
        section.append(pyo[8].text) #분실, 습득, 학생처습득 세 종류임
        process.append(pyo[9].text)
        phonenum_yn.append(pyo[10].text)
        phonenum.append(pyo[11].text)
        place.append(pyo[12].text)
        place_details.append(pyo[13].text)
        state.append(pyo[14].text)
        text.append(driver.find_element(By.CLASS_NAME,'pd-0').text)

        try:
            img=driver.find_element(By.CLASS_NAME,'pd-0').find_element(By.CLASS_NAME,'wp100.mgb-10').find_element(By.TAG_NAME,'img')
            image.append(img.get_attribute('data-url'))
            driver.back()

        except:
            image.append('no image')
            driver.back()

    print("*************complete*************")

## 다음 페이지 이동

In [ ]:
def page_move(page_num):
    n=driver.find_elements(By.CLASS_NAME, 'page-navi')

    if(page_num%10!=0):
        for i in range(2,len(n)-2):
            x=n[i].find_element(By.TAG_NAME,'a').get_attribute('title')
            if(int(x)==page_num+1):
                xx=n[i].find_element(By.TAG_NAME,'a')
                driver.execute_script('arguments[0].click();',xx)
                page_num +=1
                break

    else:
        print("*")
        n=driver.find_elements(By.CLASS_NAME, 'page-navi')
        xx=n[2].find_element(By.TAG_NAME,'a'); driver.execute_script('arguments[0].click();',xx)
        time.sleep(3)
        n=driver.find_elements(By.CLASS_NAME, 'page-navi')
        xx=n[12].find_element(By.TAG_NAME,'a'); driver.execute_script('arguments[0].click();',xx)
        page_num +=1
    return page_num

## 실행코드

In [ ]:
page_num=1
for i in range(15):
    print("page_num", page_num)
    page_crawling()
    page_num=page_move(page_num)
    driver.implicitly_wait(1)

page_num 1
*************complete*************
page_num 2
*************complete*************
page_num 3
*************complete*************
page_num 4
*************complete*************
page_num 5
*************complete*************
page_num 6
*************complete*************
page_num 7
*************complete*************
page_num 8
*************complete*************
page_num 9
*************complete*************
page_num 10
*************complete*************
*
page_num 11
*************complete*************
page_num 12
*************complete*************
page_num 13
*************complete*************
page_num 14
*************complete*************
page_num 15
*************complete*************


## csv로 데이터 저장

In [ ]:
data = {'title': title,
        'period': period,
        'postdate' : postdate,
        'modifydate': modifydate,
        'person': person,
        'view' : view,
        'category': category,
        'details': details,
        'section' : section,
        'process': process,
        'phonenum_yn': phonenum_yn,
        'phonenum' : phonenum,
        'place': place,
        'place_details': place_details,
        'state' : state,
        'image': image,
        'text': text
       }
data = pd.DataFrame(data)

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          214 non-null    object
 1   period         214 non-null    object
 2   postdate       214 non-null    object
 3   modifydate     214 non-null    object
 4   person         214 non-null    object
 5   view           214 non-null    object
 6   category       214 non-null    object
 7   details        214 non-null    object
 8   section        214 non-null    object
 9   process        214 non-null    object
 10  phonenum_yn    214 non-null    object
 11  phonenum       214 non-null    object
 12  place          214 non-null    object
 13  place_details  214 non-null    object
 14  state          214 non-null    object
 15  image          214 non-null    object
 16  text           214 non-null    object
dtypes: object(17)
memory usage: 28.5+ KB


,title,period,postdate,modifydate,person,view,category,details,section,process,phonenum_yn,phonenum,place,place_details,state,image,text
0,2024년 5월 16일 (목) 오늘의 습득물입니다.,2024-05-16 16:39 ~ 2024-11-16 16:38,2024-05-16 16:39,,김다예,35,기타,습득물,학생처습득,습득물,공개,0232772064,직접입력,학생서비스센터,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,2024년 5월 16일 (목) 오늘의 습득물입니다.\n\n 1. 분철도서(다큐멘터...
1,2024년 5월 14일 (화) 오늘의 습득물입니다.,2024-05-14 16:38 ~ 2024-11-14 16:36,2024-05-14 16:38,2024-05-14 16:39,김다예,43,기타,,학생처습득,습득물,공개,0232772064,직접입력,학생서비스센터,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,2024년 5월 14일 (화) 오늘의 습득물입니다.\n\n 1. 노트\n\n2. ...
2,c타입 아이패드 충전기 분실,2024-05-14 10:53 ~ 2025-05-28 23:59,2024-05-14 10:53,,설수인,29,USB/MP3/휴대전화,,분실,,비공개,,직접입력,이하우스 기숙사,처리중,no image,
3,2024년 5월 13일 (월) 오늘의 습득물입니다.,2024-05-13 16:48 ~ 2024-11-13 16:43,2024-05-13 16:48,,김다예,74,기타,습득물,학생처습득,습득물,공개,0232772064,직접입력,학생서비스센터,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,2024년 5월 13일 (월) 오늘의 습득물입니다.\n\n 1. 안경(검정)\n\...
4,애플 매직키보드,2024-05-13 16:33 ~ 2024-06-13 16:32,2024-05-13 16:33,2024-05-13 16:35,안하은,34,USB/MP3/휴대전화,,분실,,공개,01041879318,중앙도서관,,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,애플 븡루투스 매직키보드 중앙도서관 pc실에서 분실했습니다...\n보신 분 연락 부...


In [ ]:
data.to_csv('bunsilmul_0517.csv', index=False,encoding="utf-8-sig")

In [ ]:
data['image']

0      https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...
1      https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...
2                                               no image
3      https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...
4      https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...
                             ...                        
209                                             no image
210                                             no image
211                                             no image
212                                             no image
213                                             no image
Name: image, Length: 214, dtype: object

## 이미지 저장

In [ ]:
import urllib.request
import time

img_dst=r"C:\Users\seoyounglee\workspace\졸프\image/" #각자 경로 적고 image라는 폴더를 만들면 거기에 저장됨
for i in range(len(data)):
    url=data['image'][i]
    if(url != "no image"): urllib.request.urlretrieve(url, img_dst+str(i+1)+'.jpg')
    #img = Image.open(name)

('C:\\Users\\seoyounglee\\workspace\\졸프\\image/1.jpg',
 <http.client.HTTPMessage at 0x1d00e013550>)

('C:\\Users\\seoyounglee\\workspace\\졸프\\image/2.jpg',
 <http.client.HTTPMessage at 0x1d00e0130a0>)

('C:\\Users\\seoyounglee\\workspace\\졸프\\image/4.jpg',
 <http.client.HTTPMessage at 0x1d00e0136d0>)

('C:\\Users\\seoyounglee\\workspace\\졸프\\image/5.jpg',
 <http.client.HTTPMessage at 0x1d00e013580>)

('C:\\Users\\seoyounglee\\workspace\\졸프\\image/7.jpg',
 <http.client.HTTPMessage at 0x1d00e013460>)

...

('C:\\Users\\seoyounglee\\workspace\\졸프\\image/205.jpg',
 <http.client.HTTPMessage at 0x1d00e0135b0>)

('C:\\Users\\seoyounglee\\workspace\\졸프\\image/206.jpg',
 <http.client.HTTPMessage at 0x1d00e013760>)

('C:\\Users\\seoyounglee\\workspace\\졸프\\image/207.jpg',
 <http.client.HTTPMessage at 0x1d00e0137f0>)

('C:\\Users\\seoyounglee\\workspace\\졸프\\image/208.jpg',
 <http.client.HTTPMessage at 0x1d00e0135e0>)

('C:\\Users\\seoyounglee\\workspace\\졸프\\image/209.jpg',
 <http.client.HTTPMessage at 0x1d00e013730>)

# 텍스트 전처리

In [ ]:
import cv2
import pandas as pd
import numpy as np
import nltk
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

base_url='/content/drive/MyDrive/Colab Notebooks/졸업프로젝트/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/졸업프로젝트/bunsilmul_0517.csv')
data.head()

,title,period,postdate,modifydate,person,view,category,details,section,process,phonenum_yn,phonenum,place,place_details,state,image,text
0,2024년 5월 16일 (목) 오늘의 습득물입니다.,2024-05-16 16:39 ~ 2024-11-16 16:38,2024-05-16 16:39,NaN,김다예,35,기타,습득물,학생처습득,습득물,공개,0232772064,직접입력,학생서비스센터,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,2024년 5월 16일 (목) 오늘의 습득물입니다.\n\n 1. 분철도서(다큐멘터...
1,2024년 5월 14일 (화) 오늘의 습득물입니다.,2024-05-14 16:38 ~ 2024-11-14 16:36,2024-05-14 16:38,2024-05-14 16:39,김다예,43,기타,NaN,학생처습득,습득물,공개,0232772064,직접입력,학생서비스센터,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,2024년 5월 14일 (화) 오늘의 습득물입니다.\n\n 1. 노트\n\n2. ...
2,c타입 아이패드 충전기 분실,2024-05-14 10:53 ~ 2025-05-28 23:59,2024-05-14 10:53,NaN,설수인,29,USB/MP3/휴대전화,NaN,분실,NaN,비공개,NaN,직접입력,이하우스 기숙사,처리중,no image,NaN
3,2024년 5월 13일 (월) 오늘의 습득물입니다.,2024-05-13 16:48 ~ 2024-11-13 16:43,2024-05-13 16:48,NaN,김다예,74,기타,습득물,학생처습득,습득물,공개,0232772064,직접입력,학생서비스센터,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,2024년 5월 13일 (월) 오늘의 습득물입니다.\n\n 1. 안경(검정)\n\...
4,애플 매직키보드,2024-05-13 16:33 ~ 2024-06-13 16:32,2024-05-13 16:33,2024-05-13 16:35,안하은,34,USB/MP3/휴대전화,NaN,분실,NaN,공개,01041879318,중앙도서관,NaN,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,애플 븡루투스 매직키보드 중앙도서관 pc실에서 분실했습니다...\n보신 분 연락 부...


In [ ]:
lost_item=data[data['section']=="학생처습득"]
n=len(lost_item)
idx=lost_item.index
idx

Index([  0,   1,   3,   8,  10,  11,  12,  16,  18,  19,
       ...
       156, 157, 160, 161, 162, 163, 164, 165, 191, 205],
      dtype='int64', length=107)

## 1차 전처리_핵심 물건 텍스트만 정리

In [ ]:
n=len(lost_item)
object_list=['']*n
idx=lost_item.index

for i in range(n):
  x=lost_item['text'][idx[i]]
  xx=x.split("오늘의 습득물입니다.")[1].split("학생서비스센터")[0].replace("\n"," ")
  object_list[i]=xx

pattern = r'\([^)]*\)'
n=len(object_list)

object_list_remove=object_list.copy()

for i in range(n):
  text = re.sub(pattern=pattern, repl='', string= object_list[i])
  object_list_remove[i]=text

In [ ]:
object_list_remove[0:5]

x=pd.DataFrame({"object_text" : object_list_remove})
x.to_csv(base_url+'object_text.csv', index=False,encoding="utf-8-sig")

['    1. 분철도서  2.인생네컷  3.충전기 C타입   4.립밥  5.텀블러    ',
 '    1. 노트  2. 유선이어폰   ',
 '    1. 안경  2. 무선키보드  3. 옷  4. 모자  5. 어댑터  6. 유선이어폰  7. 무선이어폰  8. 유선이어폰     ',
 '    1. 인덱스 세트  2. 엽서 봉투 세트  3. 치약+칫솔  4. 유선 이어폰  5. 교통카드       ',
 '     1. 구찌 쇼핑백      ']

In [ ]:
text_original=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/졸업프로젝트/object_text.csv')
n=len(text_original);print(n)

text=text_original.copy()
item_list=[]

for i in range(n):
  text['object_text'][i]=re.sub(r"\s+", "", text['object_text'][i]) #불필요한 공백 제거
  text['object_text'][i]=re.sub(r'\d', '', text['object_text'][i]) #숫자제거
  text['object_text'][i]=text['object_text'][i].replace("."," ") #.제거
  text['object_text'][i]=text['object_text'][i].lower()
  item_list.append(text['object_text'][i])

item_list[0:5]
x=pd.DataFrame(item_list); print(len(x))
x.to_csv(base_url+'item_list_clean.csv', index=False,encoding="utf-8-sig")

107


[' 분철도서 인생네컷 충전기c타입 립밥 텀블러',
 ' 노트 유선이어폰',
 ' 안경 무선키보드 옷 모자 어댑터 유선이어폰 무선이어폰 유선이어폰',
 ' 인덱스세트 엽서봉투세트 치약+칫솔 유선이어폰 교통카드',
 ' 구찌쇼핑백']

107


In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/졸업프로젝트/bunsilmul_0521.csv')

data['object_list']=['']*len(data)
data['detail']=['']*len(data)

for i in range(len(x)):
  data['object_list'][idx[i]]=item_list[i]

<ipython-input-58-f229f6fd2da0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['object_list'][idx[i]]=item_list[i]
<ipython-input-58-f229f6fd2da0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['object_list'][idx[i]]=item_list[i]
<ipython-input-58-f229f6fd2da0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['object_list'][idx[i]]=item_list[i]
<ipython-input-58-f229f6fd2da0>:7: SettingWithCopyWarning: 
A value is tr

In [ ]:
data.head()
data.to_csv(base_url+'bunsilmul_0531.csv', index=False,encoding="utf-8-sig")

,title,period,postdate,modifydate,person,view,category,details,section,process,phonenum_yn,phonenum,place,place_details,state,image,text,object_num,object_list,detail
0,2024년 5월 16일 (목) 오늘의 습득물입니다.,2024-05-16 16:39 ~ 2024-11-16 16:38,2024-05-16 16:39,NaN,김다예,35,기타,습득물,학생처습득,습득물,공개,0232772064,직접입력,학생서비스센터,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,2024년 5월 16일 (목) 오늘의 습득물입니다.\n\n 1. 분철도서(다큐멘터...,5,분철도서 인생네컷 충전기c타입 립밥 텀블러,
1,2024년 5월 14일 (화) 오늘의 습득물입니다.,2024-05-14 16:38 ~ 2024-11-14 16:36,2024-05-14 16:38,2024-05-14 16:39,김다예,43,기타,NaN,학생처습득,습득물,공개,0232772064,직접입력,학생서비스센터,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,2024년 5월 14일 (화) 오늘의 습득물입니다.\n\n 1. 노트\n\n2. ...,2,노트 유선이어폰,
2,c타입 아이패드 충전기 분실,2024-05-14 10:53 ~ 2025-05-28 23:59,2024-05-14 10:53,NaN,설수인,29,USB/MP3/휴대전화,NaN,분실,NaN,비공개,NaN,직접입력,이하우스 기숙사,처리중,no image,NaN,0,,
3,2024년 5월 13일 (월) 오늘의 습득물입니다.,2024-05-13 16:48 ~ 2024-11-13 16:43,2024-05-13 16:48,NaN,김다예,74,기타,습득물,학생처습득,습득물,공개,0232772064,직접입력,학생서비스센터,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,2024년 5월 13일 (월) 오늘의 습득물입니다.\n\n 1. 안경(검정)\n\...,8,안경 무선키보드 옷 모자 어댑터 유선이어폰 무선이어폰 유선이어폰,
4,애플 매직키보드,2024-05-13 16:33 ~ 2024-06-13 16:32,2024-05-13 16:33,2024-05-13 16:35,안하은,34,USB/MP3/휴대전화,NaN,분실,NaN,공개,01041879318,중앙도서관,NaN,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,애플 븡루투스 매직키보드 중앙도서관 pc실에서 분실했습니다...\n보신 분 연락 부...,0,,


In [ ]:
data['object_list'][15:17]

15                           
16     무선헤드폰 애플펜슬 텀블러 물방울귀걸이개
Name: object_list, dtype: object

## 물건 개수 변수 추가

In [ ]:
#물건 개수
object=[]
for i in range(len(lost_item)):
  object_num=int(re.findall("\d+\.+",object_list_remove[i])[-1].replace(".",""))
  object.append(object_num)
  #lost_item['object_num'][idx[i]]=object_num

In [ ]:
data['object_num']= 0

In [ ]:
for i in range(len(object)):
  data['object_num'][idx[i]]=object[i]

<ipython-input-402-d1cd6fe9c3c4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['object_num'][idx[i]]=object[i]
<ipython-input-402-d1cd6fe9c3c4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['object_num'][idx[i]]=object[i]
<ipython-input-402-d1cd6fe9c3c4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['object_num'][idx[i]]=object[i]
<ipython-input-402-d1cd6fe9c3c4>:2: SettingWithCopyWarning: 
A value is trying to 

In [ ]:
sum(data['object_num'][idx]==object)

107

In [ ]:
data['object_num']

0      5
1      2
2      0
3      8
4      0
      ..
209    0
210    0
211    0
212    0
213    0
Name: object_num, Length: 214, dtype: int64

In [ ]:
data.to_csv(base_url+'bunsilmul_0521.csv', index=False,encoding="utf-8-sig")

## NLTK, Okt 등의 라이브러리를 통해 중요 단어 추출 시도

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from nltk.tag import pos_tag


nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#object_list_remove
for i in range(len(object_list_remove)):
  object_list_remove[i]=re.sub(r'\d', '', object_list_remove[i]).replace(".","")

pos_tag(word_tokenize(object_list_remove[0]))
word_tokenize(object_list_remove[0])

[('분철도서', 'JJ'),
 ('인생네컷', 'NNP'),
 ('충전기', 'NNP'),
 ('C타입', 'NNP'),
 ('립밥', 'NNP'),
 ('텀블러', 'NN')]

['분철도서', '인생네컷', '충전기', 'C타입', '립밥', '텀블러']

In [ ]:
word_dic=[]
for i in range(len(object_list_remove)):
  word_dic.append(pos_tag(word_tokenize(object_list_remove[i])))

word_dic[0:6]

[[('분철도서', 'JJ'),
  ('인생네컷', 'NNP'),
  ('충전기', 'NNP'),
  ('C타입', 'NNP'),
  ('립밥', 'NNP'),
  ('텀블러', 'NN')],
 [('노트', 'NN'), ('유선이어폰', 'NN')],
 [('안경', 'JJ'),
  ('무선키보드', 'NNP'),
  ('옷', 'NNP'),
  ('모자', 'NNP'),
  ('어댑터', 'NNP'),
  ('유선이어폰', 'NNP'),
  ('무선이어폰', 'NNP'),
  ('유선이어폰', 'NN')],
 [('인덱스', 'JJ'),
  ('세트', 'NNP'),
  ('엽서', 'NNP'),
  ('봉투', 'NNP'),
  ('세트', 'NNP'),
  ('치약+칫솔', 'NNP'),
  ('유선', 'NNP'),
  ('이어폰', 'NNP'),
  ('교통카드', 'NN')],
 [('구찌', 'NN'), ('쇼핑백', 'NN')],
 [('무선이어폰', 'JJ'),
  ('스타벅스카드', 'NNP'),
  ('주름', 'NNP'),
  ('종이', 'NNP'),
  ('포장지', 'NNP'),
  ('유선이어폰', 'NNP'),
  ('애플워치', 'NNP'),
  ('브러쉬', 'NNP'),
  ('USB', 'NNP'),
  ('수정테이프', 'NN')]]

모두 명사로 이루어져 있어서 class로 쓸 수 있는 단어를 추출하기가 어려움...

In [ ]:
!pip install konlpy
from konlpy.tag import Okt
from konlpy.tag import Kkma

okt = Okt()
kkma = Kkma()

okt.pos(object_list_remove[0])
okt.nouns(object_list[0]) #명사만 추출

kkma.pos(object_list_remove[0])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 34.2 MB/s eta 0:00:00


[('분철', 'Noun'),
 ('도서', 'Noun'),
 ('인생', 'Noun'),
 ('네', 'Determiner'),
 ('컷', 'Noun'),
 ('충전기', 'Noun'),
 ('C', 'Alpha'),
 ('타입', 'Noun'),
 ('립밥', 'Noun'),
 ('텀블러', 'Noun')]

['분철', '도서', '다큐멘터리', '입문', '인생', '컷', '충전기', '타입', '흰색', '립밥', '텀블러']

[('분', 'NNB'),
 ('철도', 'NNG'),
 ('서', 'JKM'),
 ('인생', 'NNG'),
 ('네', 'XSN'),
 ('컷', 'NNG'),
 ('충전기', 'NNG'),
 ('C', 'OL'),
 ('타입', 'NNG'),
 ('립', 'NNG'),
 ('밥', 'NNG'),
 ('텀블러', 'NNG')]

-> 결과가 명확하지 않아 카테고리로 일반화할 수 없었음 > 카테고라이징은 사람이 정한 기준에 따라 분류하기로 결정

## 2차 전처리_괄호 안 세부정보 변수에 저장

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/졸업프로젝트/bunsilmul_0531.csv')
data.tail(30)

,title,period,postdate,modifydate,person,view,category,details,section,process,phonenum_yn,phonenum,place,place_details,state,image,text,object_num,object_list,detail
184,약간 낡은 파란색 신용카드 찾습니다,2023-09-11 00:00 ~ 2024-09-11 23:59,2023-09-11 00:00,NaN,"Jin, Xiangxian",24,지갑/카드/신분증,NaN,분실,NaN,공개,01059299260,직접입력,학교,처리중,no image,NaN,0,NaN,NaN
185,에어팟 2세대/루이까또즈 지갑+학생증,2023-09-09 15:05 ~ 2024-09-09 23:59,2023-09-09 15:05,NaN,김효림,43,USB/MP3/휴대전화,NaN,분실,NaN,공개,01088344299,헬렌관,NaN,처리중,no image,에어팟 2세대-투명 케이스에 분홍색 키링\n루이까또즈 지갑-현금 및 카드 4장+학생...,0,NaN,NaN
186,애플펜슬,2023-09-05 13:06 ~ 2024-09-05 12:55,2023-09-05 13:06,NaN,박영아,40,기타,전자기기,분실,NaN,공개,01099421855,약학관A동,1층 104호,처리중,no image,약학관 A동 104호 교실 뒤편에서 잃어버렸습니다ㅜㅜ 보신 분 연락주세요!!,0,NaN,NaN
187,에어팟 2세대 왼쪽,2023-08-28 16:13 ~ 2024-09-04 23:59,2023-08-28 16:13,NaN,유금화,18,USB/MP3/휴대전화,NaN,분실,NaN,공개,01095080683,직접입력,ECC B2 라운지 부스,처리중,no image,8/17 ECC 라운지 1번/2번 부스에서 에어팟 2세대 왼쪽 분실\n제발 연락주세요.,0,NaN,NaN
188,검정색 루이까또즈 단지갑,2023-08-23 14:45 ~ 2024-08-23 23:59,2023-08-23 14:45,NaN,이신혜,20,장갑/목도리/기타의류,NaN,분실,NaN,공개,01080213654,직접입력,야간 셔틀버스 안,처리중,no image,8월 22일 야간 셔틀버스(새로운 셔틀버스) 앞자리에 지갑을 놓고 내렸습니다.......,0,NaN,NaN
189,젠틀몬스터 안경,2023-08-20 22:30 ~ 2024-08-20 23:59,2023-08-20 22:30,NaN,유은선,38,기타,안경,분실,NaN,공개,01047668045,직접입력,NaN,처리중,no image,한우리집-산학관 길에서 안경 분실했습니다. 습득한 분 연락주세요,0,NaN,NaN
190,셔틀버스(13일 오후 11시 40분 차량)에서 핸드폰 분실건,2023-08-13 00:51 ~ 2024-08-13 23:59,2023-08-13 00:51,NaN,황예나,27,USB/MP3/휴대전화,NaN,분실,NaN,공개,01022658674,직접입력,NaN,처리중,no image,"토요일(8월 13일)에 오후 11시 40분에 기숙사 방향의 셔틀버스를 탔는데, 핸드...",0,NaN,NaN
191,2023년 7월 13일 (목) 오늘의 습득물입니다.,2023-07-13 16:46 ~ 2025-01-13 16:40,2023-07-13 16:46,NaN,김다예,62,기타,NaN,학생처습득,NaN,공개,NaN,직접입력,학생서비스센터,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,2023년 7월 13일 (목) 오늘의 습득물입니다.\n \n \n1. 사진(포토이즘...,1,사진,NaN
192,핑크색 소세지 필통 파우치 찾습니다.,2023-07-08 18:49 ~ 2024-07-08 23:59,2023-07-08 18:49,2023-07-08 18:51,김채원,32,책/파일/필통,NaN,분실,NaN,공개,01027320973,조형예술관A동,NaN,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,습득 시 바로 전화번호로 문자로 연락주시길 바랍니다..ㅜㅜ 감사합니다.,0,NaN,NaN
193,"무선이어폰(에어팟 프로, 흰색)찾습니다.",2023-06-14 16:15 ~ 2024-06-20 23:59,2023-06-14 16:15,NaN,"GENG, YINGJIE",39,USB/MP3/휴대전화,NaN,분실,NaN,공개,01072059822,이화포스코관,NaN,처리중,no image,"안녕하세요.\n6월 13일 무선이어폰(에어팟 프로, 흰색)를 찾습니다. \n구체적인...",0,NaN,NaN


In [ ]:
for i in range(len(data)):
  if(data['section'][i]!="학생처습득"): data['object_num'][i]=1

<ipython-input-415-69cf033cde41>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if(data['section'][i]!="학생처습득"): data['object_num'][i]=1
<ipython-input-415-69cf033cde41>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if(data['section'][i]!="학생처습득"): data['object_num'][i]=1
<ipython-input-415-69cf033cde41>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if(data['section'][i]!="학생처습득"): data['object_num'][i]=1
<ipython-input-415-69c

In [ ]:
data['object_num'].unique()

array([ 5,  2,  1,  8,  9,  6, 13, 11,  4,  3, 10,  7, 14, 20])

In [ ]:
data['detail']=['']*len(data)

In [ ]:
for i in range(len(data)):
  n=data['object_num'][i]
  data['detail'][i]=[['' for col in range(1)] for row in range(n)]

<ipython-input-429-1a116853f0ff>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['detail'][i]=[['' for col in range(1)] for row in range(n)]
<ipython-input-429-1a116853f0ff>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['detail'][i]=[['' for col in range(1)] for row in range(n)]
<ipython-input-429-1a116853f0ff>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['detail'][i]=[['' for col in range(1)] for row in range(n)

In [ ]:
pattern = re.compile(r'(\d+)\.[^()]*\(([^)]+)\)')
lost_item=data[data['section']=="학생처습득"]
d=[['' for col in range(1)] for row in range(len(lost_item))]

for i in range(len(lost_item)):
  x=data['text'][idx[i]]
  x=x.split("오늘의 습득물입니다.")[1].split("학생서비스센터")[0].split("\n\n")
  #print(x)

  for j in x:
    matches = pattern.findall(j)
    if(matches != []):
      d[i] += matches

for i in range(len(lost_item)):
  d[i]=d[i][1:]

In [ ]:
d[0:5]

[[('1', '다큐멘터리 입문'), ('3', '흰색')],
 [('2', '애플, 8핀')],
 [('1', '검정'),
  ('2', '애플, 흰색'),
  ('3', '이화학잠'),
  ('4', '검정'),
  ('5', '회색'),
  ('6', '흰색, 8핀'),
  ('7', '에어팟, 왼쪽'),
  ('8', '애플, 8핀')],
 [('5', '캐시비')],
 [('1', '노트,에코백')]]

In [ ]:
z=data['detail'].copy()
idx=lost_item.index

for i in range(len(d)):
  n=len(d[i])

  for j in range(n):
    index=int(d[i][j][0])-1
    z[idx[i]][index].append(d[i][j][1])

In [ ]:
data[['text','detail']][1:20]

,text,detail
1,2024년 5월 14일 (화) 오늘의 습득물입니다.\n\n 1. 노트\n\n2. ...,"[[], [, 애플, 8핀]]"
2,NaN,[[]]
3,2024년 5월 13일 (월) 오늘의 습득물입니다.\n\n 1. 안경(검정)\n\...,"[[, 검정], [, 애플, 흰색], [, 이화학잠], [, 검정], [, 회색],..."
4,애플 븡루투스 매직키보드 중앙도서관 pc실에서 분실했습니다...\n보신 분 연락 부...,[[]]
5,이대 중앙도서관/ECC 에서 열쇠 보시면 꼭 연락 부탁드립니다...,[[]]
6,마지막 지갑을 본 기억이 ECC 잉여계단입니다ㅜㅜ 잉여계단 아니면 이씨씨~학문관~생...,[[]]
7,5/10 학문관에서 무인양품 케이블 케이스에 담겨있는 애플워치 충전기를 분실했습니다...,[[]]
8,2024년 5월 10일 (금) 오늘의 습득물입니다.\n\n 1. 인덱스 세트\n\...,"[[], [], [], [], [, 캐시비]]"
9,"이미지와 같이 생겼지만 빛깔은 사진보다 좀더 초록색이 돌고, 뚜껑이 목재입니다.",[[]]
10,2024년 5월 9일 (목) 오늘의 습득물입니다.\n\n \n 1. 구찌 쇼핑백(노...,"[[, 노트,에코백]]"


In [ ]:
data.to_csv(base_url+'bunsilmul_0602.csv', index=False,encoding="utf-8-sig")

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/졸업프로젝트/bunsilmul_0602.csv')

import ast
for i in range(len(data)):
  data['detail'][i]=data['detail'][i].replace("'',","")
  data['detail'][i]=ast.literal_eval(data['detail'][i])

data.to_csv(base_url+'bunsilmul_0603.csv', index=False,encoding="utf-8-sig")

<ipython-input-70-9991c251d3ef>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['detail'][i]=data['detail'][i].replace("'',","")
<ipython-input-70-9991c251d3ef>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['detail'][i]=ast.literal_eval(data['detail'][i])
<ipython-input-70-9991c251d3ef>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['detail'][i]=data['detail'][i].replace("'',","")
<ipython-input-70-9991c251d3ef>:7:

In [ ]:
data.to_csv(base_url+'bunsilmul_0603.csv', index=False,encoding="utf-8-sig")

## 카테고리

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/졸업프로젝트/bunsilmul_0603.csv')
data.head()

,title,period,postdate,modifydate,person,view,category,details,section,process,phonenum_yn,phonenum,place,place_details,state,image,text,object_num,object_list,detail
0,2024년 5월 16일 (목) 오늘의 습득물입니다.,2024-05-16 16:39 ~ 2024-11-16 16:38,2024-05-16 16:39,NaN,김다예,35,기타,습득물,학생처습득,습득물,공개,0232772064,직접입력,학생서비스센터,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,2024년 5월 16일 (목) 오늘의 습득물입니다.\n\n 1. 분철도서(다큐멘터...,5,분철도서 인생네컷 충전기c타입 립밥 텀블러,"[['다큐멘터리 입문'], [''], ['흰색'], [''], ['']]"
1,2024년 5월 14일 (화) 오늘의 습득물입니다.,2024-05-14 16:38 ~ 2024-11-14 16:36,2024-05-14 16:38,2024-05-14 16:39,김다예,43,기타,NaN,학생처습득,습득물,공개,0232772064,직접입력,학생서비스센터,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,2024년 5월 14일 (화) 오늘의 습득물입니다.\n\n 1. 노트\n\n2. ...,2,노트 유선이어폰,"[[''], ['애플, 8핀']]"
2,c타입 아이패드 충전기 분실,2024-05-14 10:53 ~ 2025-05-28 23:59,2024-05-14 10:53,NaN,설수인,29,USB/MP3/휴대전화,NaN,분실,NaN,비공개,NaN,직접입력,이하우스 기숙사,처리중,no image,NaN,1,NaN,[['']]
3,2024년 5월 13일 (월) 오늘의 습득물입니다.,2024-05-13 16:48 ~ 2024-11-13 16:43,2024-05-13 16:48,NaN,김다예,74,기타,습득물,학생처습득,습득물,공개,0232772064,직접입력,학생서비스센터,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,2024년 5월 13일 (월) 오늘의 습득물입니다.\n\n 1. 안경(검정)\n\...,8,안경 무선키보드 옷 모자 어댑터 유선이어폰 무선이어폰 유선이어폰,"[['검정'], ['애플, 흰색'], ['이화학잠'], ['검정'], ['회색'],..."
4,애플 매직키보드,2024-05-13 16:33 ~ 2024-06-13 16:32,2024-05-13 16:33,2024-05-13 16:35,안하은,34,USB/MP3/휴대전화,NaN,분실,NaN,공개,01041879318,중앙도서관,NaN,처리중,https://ewportal.ewha.ac.kr/workspace/bbs/BD4e...,애플 븡루투스 매직키보드 중앙도서관 pc실에서 분실했습니다...\n보신 분 연락 부...,1,NaN,[['']]


In [ ]:
!pip install webcolors

In [ ]:
  '''
  import webcolors

# 한국어 색상명을 영어로 변환
print(webcolors.name_to_rgb('빨강')) # (255, 0, 0)
print(webcolors.rgb_to_name((255, 0, 0))) # 'red'

# 16진수 색상 코드와 변환
print(webcolors.name_to_hex('파란색')) # '#0000ff'
print(webcolors.hex_to_name('#0000ff')) # 'blue'
  '''